In [1]:
import pandas as pd
import numpy as np
import math
from datetime import datetime

In [2]:
monthly_cols = []
for x in pd.read_csv('datasets/nenana_climate/1.csv').columns:
    if x.startswith('Monthly'): 
        monthly_cols.append(x)
monthly_cols

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (3,21,27,28,29,30,31,32,33,37,38,42,43,47,50,52,57,59,60,61,63,64,65,68,69,81,82,86,96,113,115,118,120) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


['MonthlyAverageRH',
 'MonthlyDaysWithGT001Precip',
 'MonthlyDaysWithGT010Precip',
 'MonthlyDaysWithGT32Temp',
 'MonthlyDaysWithGT90Temp',
 'MonthlyDaysWithLT0Temp',
 'MonthlyDaysWithLT32Temp',
 'MonthlyDepartureFromNormalAverageTemperature',
 'MonthlyDepartureFromNormalCoolingDegreeDays',
 'MonthlyDepartureFromNormalHeatingDegreeDays',
 'MonthlyDepartureFromNormalMaximumTemperature',
 'MonthlyDepartureFromNormalMinimumTemperature',
 'MonthlyDepartureFromNormalPrecipitation',
 'MonthlyDewpointTemperature',
 'MonthlyGreatestPrecip',
 'MonthlyGreatestPrecipDate',
 'MonthlyGreatestSnowDepth',
 'MonthlyGreatestSnowDepthDate',
 'MonthlyGreatestSnowfall',
 'MonthlyGreatestSnowfallDate',
 'MonthlyMaxSeaLevelPressureValue',
 'MonthlyMaxSeaLevelPressureValueDate',
 'MonthlyMaxSeaLevelPressureValueTime',
 'MonthlyMaximumTemperature',
 'MonthlyMeanTemperature',
 'MonthlyMinSeaLevelPressureValue',
 'MonthlyMinSeaLevelPressureValueDate',
 'MonthlyMinSeaLevelPressureValueTime',
 'MonthlyMinimumTempe

In [3]:
cols = [
 'DATE', 
 'REPORT_TYPE',
 'MonthlyMaximumTemperature',
 'MonthlyMeanTemperature',
 'MonthlyMinimumTemperature',
 'MonthlyStationPressure']

In [4]:
df1 = pd.read_csv('datasets/nenana_climate/1.csv', usecols=cols) 
df2 = pd.read_csv('datasets/nenana_climate/2.csv', usecols= cols) 
df3 = pd.read_csv('datasets/nenana_climate/3.csv', usecols= cols) 

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (81,82,86) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
df1 = df1[df1['REPORT_TYPE'] == 'SOM  '].dropna()
df2 = df2[df2['REPORT_TYPE'] == 'SOM  '].dropna()
df3 = df3[df3['REPORT_TYPE'] == 'SOM  '].dropna()

In [6]:
monthlies = pd.concat([df1, df2,df3])

In [7]:
shortened_date = lambda d : datetime.strptime(d[:10], '%Y-%m-%d')
month = lambda d : d.month
year = lambda d : d.year

In [8]:
# to fix some numerical values saved as strings; some end with 's'
remove_s = lambda n : (float(n[:-1]) if n.endswith('s') else float(n)) if type(n)== str else n

In [9]:
for x in ['MonthlyMaximumTemperature', 'MonthlyMeanTemperature', 'MonthlyMinimumTemperature', 'MonthlyStationPressure']:
    monthlies[x] = monthlies[x].apply(remove_s)


In [10]:
monthlies['DATE'] = monthlies['DATE'].apply(shortened_date)
monthlies['x_month'] = monthlies['DATE'].apply(month)
monthlies['Year'] = monthlies['DATE'].apply(year)

In [11]:
monthlies = monthlies.reset_index(drop=True)
monthlies = monthlies.drop(['REPORT_TYPE'], axis=1)

In [12]:
monthlies

,DATE,MonthlyMaximumTemperature,MonthlyMeanTemperature,MonthlyMinimumTemperature,MonthlyStationPressure,x_month,Year
0,2013-01-31,12.2,3.4,-5.4,29.46,1,2013
1,2013-02-28,9.4,-2.0,-13.3,29.24,2,2013
2,2013-03-31,20.2,6.0,-8.2,29.63,3,2013
3,2013-04-30,29.7,17.7,5.6,29.72,4,2013
4,2013-05-31,54.5,42.3,30.0,29.53,5,2013
...,...,...,...,...,...,...,...
256,2002-07-31,70.9,60.1,49.3,29.45,7,2002
257,2002-08-31,58.6,49.9,41.2,29.42,8,2002
258,2002-09-30,55.4,46.0,36.5,29.31,9,2002
259,2002-10-31,37.5,31.8,26.0,29.34,10,2002


In [13]:
import sqlite3
conn = sqlite3.connect('database.db')
cur = conn.cursor()
monthlies.to_sql('monthly_climate', conn, if_exists='replace', index = False)

In [14]:
labels = pd.read_sql("""
    SELECT * 
    from nib;
    """, con=conn)
labels
monthly_numerical_features = pd.read_sql("""
    SELECT * 
    from monthly_climate
    inner join nib using(Year);
    """, con=conn)
monthly_numerical_features

,DATE,MonthlyMaximumTemperature,MonthlyMeanTemperature,MonthlyMinimumTemperature,MonthlyStationPressure,x_month,Year,Decimal Day of Year,Date Time,month,TOD,TOY
0,2013-01-31 00:00:00,12.2,3.4,-5.4,29.46,1,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
1,2013-02-28 00:00:00,9.4,-2.0,-13.3,29.24,2,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
2,2013-03-31 00:00:00,20.2,6.0,-8.2,29.63,3,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
3,2013-04-30 00:00:00,29.7,17.7,5.6,29.72,4,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
4,2013-05-31 00:00:00,54.5,42.3,30.0,29.53,5,2013,140.6118,05/20/2013 02:41:00 PM,05,0.6118,140.0
...,...,...,...,...,...,...,...,...,...,...,...,...
244,2002-07-31 00:00:00,70.9,60.1,49.3,29.45,7,2002,127.8938,05/07/2002 09:27:00 PM,05,0.8938,127.0
245,2002-08-31 00:00:00,58.6,49.9,41.2,29.42,8,2002,127.8938,05/07/2002 09:27:00 PM,05,0.8938,127.0
246,2002-09-30 00:00:00,55.4,46.0,36.5,29.31,9,2002,127.8938,05/07/2002 09:27:00 PM,05,0.8938,127.0
247,2002-10-31 00:00:00,37.5,31.8,26.0,29.34,10,2002,127.8938,05/07/2002 09:27:00 PM,05,0.8938,127.0


In [15]:
monthly_numerical_features.columns = ['DATE', 'max_temp', 'mean_temp',
       'min_temp', 'station_pressure', 'x_month',
       'Year', 'decimal_day_of_year', 'date_time', 'class_month', 'TOD', 'TOY']

In [16]:
monthly_numerical_features = monthly_numerical_features.drop(
    ['DATE', 'decimal_day_of_year', 'date_time', 'TOD', 'TOY'], axis=1)
monthly_numerical_features

,max_temp,mean_temp,min_temp,station_pressure,x_month,Year,class_month
0,12.2,3.4,-5.4,29.46,1,2013,05
1,9.4,-2.0,-13.3,29.24,2,2013,05
2,20.2,6.0,-8.2,29.63,3,2013,05
3,29.7,17.7,5.6,29.72,4,2013,05
4,54.5,42.3,30.0,29.53,5,2013,05
...,...,...,...,...,...,...,...
244,70.9,60.1,49.3,29.45,7,2002,05
245,58.6,49.9,41.2,29.42,8,2002,05
246,55.4,46.0,36.5,29.31,9,2002,05
247,37.5,31.8,26.0,29.34,10,2002,05


In [16]:
monthly_numerical_features = monthly_numerical_features.sort_values('Year').reset_index(drop=True)
monthly_numerical_features.to_csv('csv/monthly_numerical_features.csv', index=False)

In [17]:
monthly_numerical_features

,DATE,max_temp,mean_temp,min_temp,station_pressure,x_month,Year,decimal_day_of_year,date_time,class_month,TOD,TOY
0,1998-05-31 00:00:00,60.5,48.6,36.6,29.27,5,1998,110.7042,04/20/1998 04:54:00 PM,04,0.7042,110.0
1,1998-10-31 00:00:00,34.3,26.0,17.6,29.27,10,1998,110.7042,04/20/1998 04:54:00 PM,04,0.7042,110.0
2,1998-09-30 00:00:00,53.9,45.1,36.2,29.28,9,1998,110.7042,04/20/1998 04:54:00 PM,04,0.7042,110.0
3,1998-07-31 00:00:00,70.5,60.5,50.4,29.44,7,1998,110.7042,04/20/1998 04:54:00 PM,04,0.7042,110.0
4,1998-11-30 00:00:00,16.9,9.0,1.1,29.21,11,1998,110.7042,04/20/1998 04:54:00 PM,04,0.7042,110.0
...,...,...,...,...,...,...,...,...,...,...,...,...
244,2020-02-29 00:00:00,8.7,-3.8,-16.3,29.42,2,2020,118.5389,04/27/2020 12:56:00 PM,04,0.5389,118.0
245,2020-01-31 00:00:00,-8.3,-16.1,-23.8,29.49,1,2020,118.5389,04/27/2020 12:56:00 PM,04,0.5389,118.0
246,2020-06-30 00:00:00,71.0,59.9,48.7,29.36,6,2020,118.5389,04/27/2020 12:56:00 PM,04,0.5389,118.0
247,2020-05-31 00:00:00,66.8,53.2,39.5,29.47,5,2020,118.5389,04/27/2020 12:56:00 PM,04,0.5389,118.0
